# Fetal Health Classification
### Justin Farnsworth (farnswj1@tcnj.edu)

In [1]:
# Make PySpark importable
import findspark
findspark.init()

In [2]:
# Imported libraries
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, RobustScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col

In [3]:
# Set up the Spark session
spark = SparkSession.builder.appName("Fetal Health Classification").getOrCreate()
sc = spark.sparkContext

In [4]:
# Load the dataset
df = spark.read.csv("fetal_health.csv", inferSchema=True, header=True)
df.toPandas()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,2.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,2.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,2.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,2.0


In [5]:
# Show the schema
df.printSchema()

root
 |-- baseline value: double (nullable = true)
 |-- accelerations: double (nullable = true)
 |-- fetal_movement: double (nullable = true)
 |-- uterine_contractions: double (nullable = true)
 |-- light_decelerations: double (nullable = true)
 |-- severe_decelerations: double (nullable = true)
 |-- prolongued_decelerations: double (nullable = true)
 |-- abnormal_short_term_variability: double (nullable = true)
 |-- mean_value_of_short_term_variability: double (nullable = true)
 |-- percentage_of_time_with_abnormal_long_term_variability: double (nullable = true)
 |-- mean_value_of_long_term_variability: double (nullable = true)
 |-- histogram_width: double (nullable = true)
 |-- histogram_min: double (nullable = true)
 |-- histogram_max: double (nullable = true)
 |-- histogram_number_of_peaks: double (nullable = true)
 |-- histogram_number_of_zeroes: double (nullable = true)
 |-- histogram_mode: double (nullable = true)
 |-- histogram_mean: double (nullable = true)
 |-- histogram_medi

In [6]:
# Check for null values
df.toPandas().isnull().sum()

baseline value                                            0
accelerations                                             0
fetal_movement                                            0
uterine_contractions                                      0
light_decelerations                                       0
severe_decelerations                                      0
prolongued_decelerations                                  0
abnormal_short_term_variability                           0
mean_value_of_short_term_variability                      0
percentage_of_time_with_abnormal_long_term_variability    0
mean_value_of_long_term_variability                       0
histogram_width                                           0
histogram_min                                             0
histogram_max                                             0
histogram_number_of_peaks                                 0
histogram_number_of_zeroes                                0
histogram_mode                          

In [7]:
# Show more information about the dataset
df.toPandas().describe()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
count,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.00000,...,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000
mean,133.303857,0.003178,0.009481,0.004366,0.001889,0.000003,0.000159,46.990122,1.332785,9.84666,...,93.579492,164.025400,4.068203,0.323612,137.452023,134.610536,138.090310,18.808090,0.320320,1.304327
std,9.840844,0.003866,0.046666,0.002946,0.002960,0.000057,0.000590,17.192814,0.883241,18.39688,...,29.560212,17.944183,2.949386,0.706059,16.381289,15.593596,14.466589,28.977636,0.610829,0.614377
min,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.200000,0.00000,...,50.000000,122.000000,0.000000,0.000000,60.000000,73.000000,77.000000,0.000000,-1.000000,1.000000
25%,126.000000,0.000000,0.000000,0.002000,0.000000,0.000000,0.000000,32.000000,0.700000,0.00000,...,67.000000,152.000000,2.000000,0.000000,129.000000,125.000000,129.000000,2.000000,0.000000,1.000000
50%,133.000000,0.002000,0.000000,0.004000,0.000000,0.000000,0.000000,49.000000,1.200000,0.00000,...,93.000000,162.000000,3.000000,0.000000,139.000000,136.000000,139.000000,7.000000,0.000000,1.000000
75%,140.000000,0.006000,0.003000,0.007000,0.003000,0.000000,0.000000,61.000000,1.700000,11.00000,...,120.000000,174.000000,6.000000,0.000000,148.000000,145.000000,148.000000,24.000000,1.000000,1.000000
max,160.000000,0.019000,0.481000,0.015000,0.015000,0.001000,0.005000,87.000000,7.000000,91.00000,...,159.000000,238.000000,18.000000,10.000000,187.000000,182.000000,186.000000,269.000000,1.000000,3.000000


In [8]:
# Get the counts of the labels
df.toPandas()["fetal_health"].value_counts()

1.0    1655
2.0     295
3.0     176
Name: fetal_health, dtype: int64

In [9]:
# Create a train set and a test set
train_set, test_set = df.randomSplit([0.7, 0.3], seed=3)

In [10]:
# Combine the features into a vector to feed into the ML algorithm
assembler = VectorAssembler(
    inputCols=list(filter(lambda c: c != "fetal_health", df.columns)),
    outputCol="features"
)

In [11]:
# Scale the features
scaler = RobustScaler(inputCol="features", outputCol="scaled_features")

In [12]:
# Create the random forest classifier
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="fetal_health")

In [13]:
# Make a param grid
grid = ParamGridBuilder()\
       .addGrid(rf.maxDepth, [2, 5, 8])\
       .addGrid(rf.maxBins, [5, 10, 15, 20])\
       .addGrid(rf.numTrees, [10, 30, 50])\
       .build()

In [14]:
# Use the evaluator to measure the performance of the model
evaluator = BinaryClassificationEvaluator(labelCol="fetal_health", rawPredictionCol="prediction")

In [15]:
# Set up cross-validation
cv = CrossValidator(estimator=rf, estimatorParamMaps=grid, evaluator=evaluator, numFolds=5)

In [16]:
# Create a pipeline
pipeline = Pipeline(stages=[assembler, scaler, cv])

In [17]:
# Train the model (this could take several minutes)
model = pipeline.fit(train_set)

In [18]:
# Make the predictions
predictions = model.transform(test_set).select(col("fetal_health"), col("prediction"))
predictions.toPandas()

,fetal_health,prediction
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
...,...,...
691,2.0,2.0
692,2.0,1.0
693,2.0,1.0
694,1.0,1.0


In [19]:
# Compute the accuracy
evaluator.evaluate(predictions)

1.0

In [20]:
# Show the confusion matrix
metrics = MulticlassMetrics(predictions.rdd)
metrics.confusionMatrix().toArray()

array([[544.,  68.],
       [  3.,  26.]])

In [21]:
# Terminate the Spark session
spark.stop()